In [1]:
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
%matplotlib inline

In [2]:
m = loadmat('../data/TrainingSamplesDCT_8_new.mat')
foreground,background = m['TrainsampleDCT_FG'],m['TrainsampleDCT_BG']

In [3]:
# define the zigzag transformation
zig_zag = np.array([[0,1,5,6,14,15,27,28],[2,4,7,13,16,26,29,42],[3,8,12,17,25,30,41,43],
                   [9,11,18,24,31,40,44,53],[10,19,23,32,39,45,52,54],[20,22,33,38,46,51,55,60],
                   [21,34,37,47,50,56,59,61],[35,36,48,49,57,58,62,63]])
zz_flat = zig_zag.flatten()
def zig_zag_transform(a):
    result = np.zeros(64)
    for i in range(64):
        result[zz_flat[i]] = a[i]
    return result

In [4]:
# 2D DCT function
import scipy.fftpack
def dct2d(a):
    return scipy.fftpack.dct(scipy.fftpack.dct( a, axis=0, norm='ortho' ),axis=1,norm='ortho')

In [5]:
im = loadmat('../data/im_double.mat')
im_array = im['img']
print(im_array.shape)

(255, 270)


In [6]:
import imageio
# store the test data as a numpy array
im_test = imageio.imread('../data/cheetah_mask.bmp')
im_test_array = np.array(im_test)
# convert 255 to 1 for error calculation
im_test_array = im_test_array / 255

In [7]:
M,C,N_FG,N_BG = 5,8,foreground.shape[0],background.shape[0]

In [183]:
def rand_init(c,sample):
    pi = np.ones(c) * 1 / c
#     mu = sample[np.random.randint(sample.shape[0],size = c),:]
    mu = np.zeros((c,64))
    cov = []
    for i in range(c):
        cov_temp = np.random.normal(5,0.3,size=64)
        cov.append(np.diag(cov_temp))
    cov = np.array(cov)
    return pi,mu,cov

In [192]:
def EM(c,sample,max_iter):
    pi,mu,cov = rand_init(c,sample)
    for i in range(max_iter):
        # E-step
        H = []
        for j in range(c):
            H_temp = multivariate_normal.pdf(sample,mean=mu[j,:],cov=cov[j,:,:]) * pi[j]
            H.append(H_temp)
        H = np.array(H).T
        H = H / np.sum(H,axis = 1)[:,np.newaxis]
        H_sum = np.sum(H,axis = 0)
        # M-step
        # update pi
        pi = 1 / sample.shape[0] * H_sum
        # update mean
        mu_update = []
        for j in range(c):
            mu_temp = np.sum(H[:,j][:,np.newaxis] * sample,axis = 0) / H_sum[j]
            mu_update.append(mu_temp)
        # update covariance
        cov_update = []
        for j in range(c):
            x_temp = sample - mu[j,:]
            cov_temp = np.sum((x_temp ** 2) * H[:,j][:,np.newaxis],axis = 0) / H_sum[j]
            # make sure cov is not too small
            cov_temp[cov_temp < 1e-6] = 1e-6
            cov_temp = np.diag(cov_temp)
            cov_update.append(cov_temp)
        cov = np.array(cov_update)
        mu = np.array(mu_update)
#         print(likelihood(pi,mu,cov,H,sample))
    return pi,mu,cov,H

In [191]:
likelihood(pi_FG,mu_FG,cov_FG,H,foreground)

36995.17187000675

In [199]:
def likelihood(pi,mu,cov,H,sample):
    result = 0
    for j in range(pi.shape[0]):
        density = multivariate_normal.pdf(sample,mean=mu[j,:],cov=cov[j,:,:]) * pi[j]
        for i in range(sample.shape[0]):
            if density[i] < 1e-4:
                continue
            result += np.log(density[i]) * H[i][j]
    return result